### Import modules and verify they work? 

In [1]:
# general python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import numpy as np
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import scipy
import xarray as xr
from tqdm import tqdm
import glob
from devtools import pprint
from tqdm import tqdm

In [2]:
# general eWC
import ewatercycle
import ewatercycle.forcing
import ewatercycle.models

Download plugin model

In [3]:
# pip install ewatercycle-lorenz

#### set up paths

In [4]:
path = Path.cwd()
forcing_path = path / "Forcing"
observations_path = path / "Observations"
figure_path = path / "Figures"
output_path = path / "Output"
forcing_path

PosixPath('/home/davidhaasnoot/eWaterCycle-WSL-WIP/Forcing')

#### add parameter info

In [5]:
experiment_start_date = "1997-10-01T00:00:00Z"
experiment_end_date = "1997-10-10T00:00:00Z"

In [6]:
from ewatercycle.forcing import sources

In [7]:
lorenz_forcing = sources.LorenzForcing(start_time = experiment_start_date,
                                      end_time = experiment_end_date,
                                      directory = forcing_path,
                                      F = 8,
                                      dt=1e-3
                          )

#### import model

In [8]:
from ewatercycle.models import Lorenz

#### import DA function:

In [9]:
import importlib.util

def module_from_file(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

In [10]:
DA = module_from_file("DA",r'../eWaterCycle-DA/src/ewatercycle_DA/DA.py')

In [11]:
n_particles = 20

In [12]:
ensemble = DA.Ensemble(N=n_particles)
ensemble.setup()

In [13]:
J = 40
common_state = np.zeros(J)
common_state[19] = 0.01

### petrub state every so slightly to start with:

In [17]:
rng = np.random.default_rng()
def add_normal_noise(like_sigma) -> float:
    """Normal (zero-mean) noise to be added to a state

    Args:
        like_sigma (float): scale parameter - pseudo variance & thus 'like'-sigma

    Returns:
        sample from normal distribution
    """
    return rng.normal(loc=0, scale=like_sigma)  # log normal so can't go to 0 ?

In [18]:
# values wihch you 
setup_kwargs_lst = []
for index in range(n_particles):
    peturbed_state = common_state.copy()
    peturbed_state[5] += add_normal_noise(0.01)
    setup_kwargs_lst.append({'J':J, 
                            'start_state':list(peturbed_state),
                             })

In [22]:
ensemble.initialize(model_name=["Lorenz"]*n_particles,
                    forcing=[lorenz_forcing]*n_particles,
                    setup_kwargs=setup_kwargs_lst)

In [23]:
pprint(ensemble.ensemble_list)

[
    EnsembleMember(
        model_name='Lorenz',
        forcing=LorenzForcing(
            start_time='1997-10-01T00:00:00Z',
            end_time='1997-10-10T00:00:00Z',
            directory=PosixPath('/home/davidhaasnoot/eWaterCycle-WSL-WIP/Forcing'),
            shape=None,
            filenames={},
            F=8.0,
            dt=0.001,
        ),
        setup_kwargs={
            'J': 40,
            'start_state': [
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.002217827984405591,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.01,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
                0.0,
   

if fails to initialize, run in cmd:
[link1](https://stackoverflow.com/questions/65272764/ports-are-not-available-listen-tcp-0-0-0-0-50070-bind-an-attempt-was-made-to)
[link2](https://asheroto.medium.com/docker-error-an-attempt-was-made-to-access-a-socket-in-a-way-forbidden-by-its-access-permissions-15a444ab217b)
```bash
net stop winnat
netsh int ipv4 set dynamic tcp start=49152 num=16384
netsh int ipv6 set dynamic tcp start=49152 num=16384
net start winnat
````

`%SystemRoot%\system32\WindowsPowerShell\v1.0\powershell.exe`

In [24]:
# # #### run if fails 
# ensemble.finalize()

## run truth model as observations

In [25]:
truth_model = Lorenz(forcing=lorenz_forcing)
config, _ = truth_model.setup(J=J,
                        start_state=list(common_state))
truth_model.initialize(config)

In [26]:
n_timesteps = int(round((truth_model.end_time - truth_model.start_time) /  truth_model.time_step,0))

output = pd.DataFrame(columns =['truth'])
output_lst = []

for _ in tqdm(range(n_timesteps)):   
    truth_model.update()
    output.loc[truth_model.time_as_datetime] = truth_model.get_value("state")[5]
    output_lst.append(truth_model.get_value("state"))
truth_model.finalize()

100%|██████████████████████████████████████████████████████████████████████████████| 9000/9000 [00:42<00:00, 212.65it/s]


In [27]:
output.index.name = "time"
current_time = str(datetime.now())[:-10].replace(":","_")
ds_obs_dir = observations_path / f'truth_model_lorenz96_{current_time}.nc'
# ds_obs = xr.Dataset(data_vars=output[['truth']])
# ds_obs = xr.Dataset({'truth':xr.DataArray(data=pd.DataFrame(output_lst,index=output.index),dims=["time","x"])})
ds_obs = xr.Dataset({'truth':xr.DataArray(data=pd.DataFrame(output,index=output.index),dims=["time","x"])})
if not ds_obs_dir.exists():
    ds_obs.to_netcdf(ds_obs_dir)

## setup DA

This sets up all the require data assimilation information

In [28]:
def H(Z):
    """Function which takes the state vector Z and returns part of that state. Returned should be same shape as data provided"""
    if len(Z) == J:
        return Z[5] 
    else: 
        raise SyntaxWarning(f"Length of statevector should be {J} but is {len(Z)}")

In [29]:
ensemble.initialize_da_method(ensemble_method_name = "PF", 
                           hyper_parameters = {
                                               'like_sigma_weights' : 0.05,
                                               'like_sigma_state_vector' : 2.0,
                                               },
                           state_vector_variables = ["state"],
                           observation_path = ds_obs_dir,
                           observed_variable_name = "truth",
                           measurement_operator= H                           
                            )

## Run

### Troubleshoot setting state vector

In [30]:
ref_model = ensemble.ensemble_list[0].model

In [31]:
n_timesteps = int(round((ref_model.end_time - ref_model.start_time) /  ref_model.time_step,0))
time = []
lst_state_vector = []
lst_Q = [] 
for i in tqdm(range(n_timesteps)):    
    time.append(pd.Timestamp(ref_model.time_as_datetime.date()))
    assimilate = False
    ensemble.update(assimilate)
    lst_state_vector.append(ensemble.get_state_vector())
    # lst_Q.append(ensemble.get_value("state"))
    # TODO: adjust so that tLag ? currently still often 3

# end model - IMPORTANT! when working with dockers
ensemble.finalize()

100%|███████████████████████████████████████████████████████████████████████████████| 9000/9000 [04:04<00:00, 36.81it/s]


In [ ]:
# ensemble.ensemble_method.weights.shape

In [33]:
Q_m_arr = np.array(lst_Q).T
state_vector_arr = np.array(lst_state_vector)

### process the numpy data into easily acessed data types

In [34]:
save, load = False, False
current_time = str(datetime.now())[:-10].replace(":","_")

### process states and parameters into xarrys

In [35]:
if not load:    
    data_vars = {}
    for i, name in enumerate(range(J)):
        storage_terms_i = xr.DataArray(state_vector_arr[:,:,i].T,
                                       name=name,
                                       dims=["EnsembleMember","time"],
                                      coords=[np.arange(n_particles),time[:len(state_vector_arr)]],
                                      attrs={"title": f"Lorenz states over time for {n_particles} particles ", 
                                               "history": f"states of lorenz model: ewatercycle DA",
                                            "description":"Moddeled values",
                                                 "units": "mm"})
        data_vars[name] = storage_terms_i

    ds_combined = xr.Dataset(data_vars,
                             attrs={"title": f"Lorenz model over time for {n_particles} particles ", 
                                    "history": f"states of lorenz model: ewatercycle DA",}
                              )

In [36]:
##### Save? 
if save:
    ds_combined.to_netcdf(output_path / f'combined_ds_{current_time}.nc')
    
if load:
    ds_combined = xr.open_dataset(glob.glob(str(output_path / 'combined_ds_*.nc'))[-1])

## Plotting

In [37]:
save = True

In [43]:
fig, ax = plt.subplots(1,1,figsize=(12,5))
x = 5
for i in range(n_particles):
    # ds_combined[7].isel(EnsembleMember=i).plot(ax=ax)
    plt.plot(list(output.index)[:len(ds_combined[x].isel(EnsembleMember=i).values)],ds_combined[x].isel(EnsembleMember=i).values,lw=0.5,color='C0',zorder=-1)
# for x in range(J):
ds_obs['truth'].plot(zorder=1,color="C1")
ax.set_title(f"Lorenz model without assimilation for {n_particles} ensemble members")
if save: 
    fig.savefig(figure_path / "lorenz_model_no-assimilation.png",bbox_inches="tight",dpi=400)